# Mi cadena de markov

## Bibliotecas

In [2]:
# Warnings ---------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings("ignore")

# Lectura y manipulación de datos ----------------------------------------------------------------------------

import pandas as pd

# Descomposición espectral -----------------------------------------------------------------------------------

import numpy as np
from numpy.linalg import eig, inv


## Lectura de la base de datos

In [3]:
raw_data = pd.read_parquet('data/tec_estocasticos.parquet', engine='pyarrow')

In [4]:
raw_data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


### Tipos de Cliente

In [5]:
raw_data['tipo_cliente'].unique()

array(['Distribuidor', 'Hospital', 'Farmacia', 'Otro'], dtype=object)

In [6]:
raw_data['periodo'] = pd.to_datetime(raw_data['periodo'])
raw_data.sort_values(by='periodo', inplace=True)
raw_data.dropna(inplace=True)
raw_data.reset_index(inplace=True)
raw_data.drop('index', axis = 1,inplace=True)
raw_data

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


In [7]:
hospital =raw_data.loc[raw_data['tipo_cliente'] == 'Hospital']

In [8]:
p_317 = hospital.loc[hospital['material_id'] == 317.0]

p_317

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1505,2021-01-01,2825.0,317.0,Hospital
1882,2021-01-01,1911.0,317.0,Hospital
2532,2021-01-01,1282.0,317.0,Hospital
3147,2021-01-01,1356.0,317.0,Hospital
...,...,...,...,...
6955076,2023-09-01,3505.0,317.0,Hospital
6955115,2023-09-01,799.0,317.0,Hospital
6955172,2023-09-01,4476.0,317.0,Hospital
6955296,2023-09-01,2211.0,317.0,Hospital


In [9]:
p_317.groupby('periodo').head()

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1505,2021-01-01,2825.0,317.0,Hospital
1882,2021-01-01,1911.0,317.0,Hospital
2532,2021-01-01,1282.0,317.0,Hospital
3147,2021-01-01,1356.0,317.0,Hospital
...,...,...,...,...
6715984,2023-09-01,4159.0,317.0,Hospital
6716013,2023-09-01,1423.0,317.0,Hospital
6716025,2023-09-01,1179.0,317.0,Hospital
6716059,2023-09-01,4656.0,317.0,Hospital


In [10]:
h = p_317.loc[p_317['cliente_id'] == 800.0]
h.reset_index(inplace = True)
h.drop('index', axis = 1, inplace = True)
h

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-07-01,800.0,317.0,Hospital
2,2021-09-01,800.0,317.0,Hospital
3,2021-10-01,800.0,317.0,Hospital
4,2021-11-01,800.0,317.0,Hospital
5,2022-01-01,800.0,317.0,Hospital
6,2022-02-01,800.0,317.0,Hospital
7,2022-03-01,800.0,317.0,Hospital
8,2022-04-01,800.0,317.0,Hospital
9,2022-05-01,800.0,317.0,Hospital


In [11]:
(h['periodo'][1] - h['periodo'][0]).days

181

In [12]:
181 // 30

6

In [27]:
t = [0]

for x in range(0, len(h['periodo'])-2):
    if (h['periodo'][x+1] - h['periodo'][x]).days <= 31:
        t.append(0)
    else:
        for _ in range(((h['periodo'][x+1] - h['periodo'][x]).days // 30)-1):
            t.append(1)
        t.append(0)
t

[0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0]

In [40]:
t_1 = []

for x in range(0, len(h['periodo'])-1):
    if (h['periodo'][x+1] - h['periodo'][x]).days <= 31:
        t_1.append(0)
    else:
        for _ in range(((h['periodo'][x+1] - h['periodo'][x]).days // 30)-1):
            t_1.append(1)
        t_1.append(0)
t_1

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [41]:
p = pd.DataFrame()

p['t'] = t
p['t_1'] = t_1

p

,t,t_1
0,0,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,0
6,0,1
7,1,0
8,0,0
9,0,0


In [39]:
len(t_1)

31

In [34]:
len(fechas)

33

In [58]:
fechas

[Timestamp('2021-01-01 00:00:00'),
 Timestamp('2021-02-01 00:00:00'),
 Timestamp('2021-03-01 00:00:00'),
 Timestamp('2021-04-01 00:00:00'),
 Timestamp('2021-05-01 00:00:00'),
 Timestamp('2021-06-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-08-01 00:00:00'),
 Timestamp('2021-09-01 00:00:00'),
 Timestamp('2021-10-01 00:00:00'),
 Timestamp('2021-11-01 00:00:00'),
 Timestamp('2021-12-01 00:00:00'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-02-01 00:00:00'),
 Timestamp('2022-03-01 00:00:00'),
 Timestamp('2022-04-01 00:00:00'),
 Timestamp('2022-05-01 00:00:00'),
 Timestamp('2022-06-01 00:00:00'),
 Timestamp('2022-07-01 00:00:00'),
 Timestamp('2022-08-01 00:00:00'),
 Timestamp('2022-09-01 00:00:00'),
 Timestamp('2022-10-01 00:00:00'),
 Timestamp('2022-11-01 00:00:00'),
 Timestamp('2022-12-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-02-01 00:00:00'),
 Timestamp('2023-03-01 00:00:00'),
 Timestamp('2023-04-01 00:00:00'),
 Timestamp('2023-05-

In [33]:
import pandas as pd

# Fecha de inicio
fecha_inicio = pd.Timestamp(2021, 1, 1)

# Fecha de finalización
fecha_fin = pd.Timestamp(2023, 9, 1)

# Frecuencia mensual
frecuencia = pd.DateOffset(months=1)

# Lista para almacenar las fechas
fechas = []

# Bucle para generar las fechas mensuales
while fecha_inicio <= fecha_fin:
    fechas.append(fecha_inicio)
    fecha_inicio += frecuencia

fechas

[Timestamp('2021-01-01 00:00:00'),
 Timestamp('2021-02-01 00:00:00'),
 Timestamp('2021-03-01 00:00:00'),
 Timestamp('2021-04-01 00:00:00'),
 Timestamp('2021-05-01 00:00:00'),
 Timestamp('2021-06-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-08-01 00:00:00'),
 Timestamp('2021-09-01 00:00:00'),
 Timestamp('2021-10-01 00:00:00'),
 Timestamp('2021-11-01 00:00:00'),
 Timestamp('2021-12-01 00:00:00'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-02-01 00:00:00'),
 Timestamp('2022-03-01 00:00:00'),
 Timestamp('2022-04-01 00:00:00'),
 Timestamp('2022-05-01 00:00:00'),
 Timestamp('2022-06-01 00:00:00'),
 Timestamp('2022-07-01 00:00:00'),
 Timestamp('2022-08-01 00:00:00'),
 Timestamp('2022-09-01 00:00:00'),
 Timestamp('2022-10-01 00:00:00'),
 Timestamp('2022-11-01 00:00:00'),
 Timestamp('2022-12-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-02-01 00:00:00'),
 Timestamp('2023-03-01 00:00:00'),
 Timestamp('2023-04-01 00:00:00'),
 Timestamp('2023-05-

In [ ]:
t = []
t_1 = []

for i in range(len(fechas)):
    while mes <= fecha_fin:
    fechas.append(fecha_inicio)
    fecha_inicio += frecuencia

In [13]:
len(raw_data['material_id'].unique())

1689

In [42]:
Xt= p['t'][0:-1].reset_index(drop=True).rename('X_t')
Xt_1 = p['t_1'][1::].reset_index(drop=True).rename('X_t+1')

In [43]:
new_data=pd.concat((Xt, Xt_1), axis=1)

In [44]:
matriz_transicion = new_data.groupby('X_t').value_counts(normalize=True).unstack(level='X_t+1')
matriz_transicion= matriz_transicion.fillna(0)
matriz_transicion

X_t+1,0,1
X_t,,
0,0.818182,0.181818
1,0.555556,0.444444


In [45]:
## Descomposición espectral

import numpy as np
from numpy.linalg import eig, inv
Lambda, Q = eig(matriz_transicion)
print("Los eigenvalores de P son:", Lambda)
print("Los eigenvectores de P son:", Q)

Los eigenvalores de P son: [1.         0.26262626]
Los eigenvectores de P son: [[ 0.70710678 -0.31103906]
 [ 0.70710678  0.95039713]]


In [46]:
Q_1 = inv(Q)
Lambda = np.diag(Lambda)
PP = np.matmul(np.matmul(Q, Lambda), Q_1)
PP.round(decimals = 4)

array([[0.8182, 0.1818],
       [0.5556, 0.4444]])

In [47]:
def pasos(n):
    Lambda_n = Lambda**n
    P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)
    return P_n.round(decimals = 4)

In [48]:
pasos(5)

array([[0.7537, 0.2463],
       [0.7525, 0.2475]])

In [49]:
pasos(10)

array([[0.7534, 0.2466],
       [0.7534, 0.2466]])

In [50]:
pasos(15)

array([[0.7534, 0.2466],
       [0.7534, 0.2466]])

In [22]:
pasos(70)

array([[ 9.99700000e-01,  3.00000000e-04,  0.00000000e+00, ...,
         2.16317869e+22,  3.62133347e+23, -1.33534641e+22],
       [ 0.00000000e+00,  9.99700000e-01,  3.00000000e-04, ...,
        -9.65261081e+20, -5.58997353e+22,  5.84001225e+22],
       [ 0.00000000e+00,  0.00000000e+00,  9.99700000e-01, ...,
        -2.77606162e+20, -1.59895843e+21,  1.97473608e+21],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.99100000e-01,  9.00000000e-04,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  9.99600000e-01,  4.00000000e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [23]:
pasos(75)

array([[ 9.99600000e-01,  4.00000000e-04,  0.00000000e+00, ...,
        -2.06518165e+22,  9.26123846e+21,  2.46836234e+23],
       [ 0.00000000e+00,  9.99600000e-01,  4.00000000e-04, ...,
        -2.86329607e+20, -2.19260740e+22,  5.90177614e+21],
       [ 0.00000000e+00,  0.00000000e+00,  9.99700000e-01, ...,
        -2.00404304e+20,  8.13479146e+20,  8.36384849e+20],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.99000000e-01,  1.00000000e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  9.99600000e-01,  4.00000000e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [24]:
pasos(100)

array([[ 9.99500000e-01,  5.00000000e-04,  0.00000000e+00, ...,
         2.86151151e+21,  2.07750117e+23,  3.02727090e+23],
       [ 0.00000000e+00,  9.99500000e-01,  5.00000000e-04, ...,
         1.31341891e+21, -1.90641333e+22, -2.75830580e+22],
       [ 0.00000000e+00,  0.00000000e+00,  9.99600000e-01, ...,
        -2.29001423e+20,  8.31690610e+20, -2.18168860e+21],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.98700000e-01,  1.30000000e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  9.99500000e-01,  5.00000000e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [25]:
pasos(110)

array([[ 9.99500000e-01,  5.00000000e-04,  0.00000000e+00, ...,
         2.88253347e+22,  2.00173236e+23,  9.27700083e+22],
       [ 0.00000000e+00,  9.99500000e-01,  5.00000000e-04, ...,
         2.20819546e+21,  2.29997356e+22,  4.26478690e+21],
       [ 0.00000000e+00,  0.00000000e+00,  9.99500000e-01, ...,
        -2.44033263e+19,  2.90549400e+21, -4.02318241e+21],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.98600000e-01,  1.40000000e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  9.99400000e-01,  6.00000000e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])